# 机器学习 实验六 决策树分类器

- 专业：物联网工程
- 姓名：秦泽斌
- 学号：2212005

## 一、实验要求
### 1. 基本要求
- 基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；
- 基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；
### 2. 中级要求
- 对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；
- 对测试集Watermelon-test2进行预测，输出分类精度；
### 3. 高级要求
- 使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。

## 二、实验内容

### 1.基本要求
- **基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；**

- **基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；**

#### 1.1前期准备

In [83]:
# 引入头文件
import pandas as pd
from math import log2

In [84]:
# Load Data
encoding = 'GBK' # 由于数据集是中文 需要特定的编码支持
train1_path = 'Watermelon-train1.csv'
test1_path = 'Watermelon-test1.csv'
train2_path = 'Watermelon-train2.csv'
test2_path = 'Watermelon-test2.csv'
train1_df = pd.read_csv(train1_path,encoding=encoding)
test1_df = pd.read_csv(test1_path,encoding=encoding)
train2_df = pd.read_csv(train2_path,encoding=encoding)
test2_df = pd.read_csv(test2_path,encoding=encoding)
train1_df.head(), test1_df.head(),train2_df.head(), test2_df.head()

(   编号  色泽  根蒂  敲声  纹理 好瓜
 0   1  青绿  蜷缩  浊响  清晰  是
 1   2  乌黑  蜷缩  沉闷  清晰  是
 2   3  乌黑  蜷缩  浊响  清晰  是
 3   4  青绿  蜷缩  沉闷  清晰  是
 4   5  浅白  蜷缩  浊响  清晰  是,
    编号  色泽  根蒂  敲声  纹理 好瓜
 0   1  浅白  蜷缩  浊响  清晰  是
 1   2  乌黑  稍蜷  沉闷  清晰  是
 2   3  乌黑  蜷缩  沉闷  清晰  是
 3   4  青绿  蜷缩  沉闷  稍糊  是
 4   5  浅白  蜷缩  浊响  清晰  是,
    编号  色泽  根蒂  敲声  纹理     密度 好瓜
 0   1  青绿  蜷缩  浊响  清晰  0.697  是
 1   2  乌黑  蜷缩  沉闷  清晰  0.774  是
 2   3  乌黑  蜷缩  浊响  清晰  0.634  是
 3   4  青绿  蜷缩  沉闷  清晰  0.608  是
 4   5  浅白  蜷缩  浊响  清晰  0.556  是,
    编号  色泽  根蒂  敲声  纹理     密度 好瓜
 0   1  乌黑  稍蜷  浊响  清晰  0.403  是
 1   2  青绿  稍蜷  浊响  稍糊  0.481  是
 2   3  乌黑  稍蜷  浊响  清晰  0.337  是
 3   4  乌黑  稍蜷  沉闷  稍糊  0.666  否
 4   5  青绿  硬挺  清脆  清晰  0.243  否)

In [85]:
# 计算信息增益以及信息熵 展示对应信息

def calculate_entropy(data, target):
    value_counts = data[target].value_counts()
    entropy = sum([-count/len(data) * log2(count/len(data)) for count in value_counts])
    return entropy

def calculate_information_gain(data, attribute, target):
    total_entropy = calculate_entropy(data, target)
    values = data[attribute].unique()
    weighted_entropy = sum([len(data[data[attribute] == value])/len(data) * calculate_entropy(data[data[attribute] == value], target) for value in values])
    information_gain = total_entropy - weighted_entropy
    return information_gain

target_attribute = '好瓜'
info_gains_train1 = {col: calculate_information_gain(train1_df, col, target_attribute) for col in train1_df.columns if col != target_attribute}

info_gains_train1



{'编号': 1.0,
 '色泽': 0.17379494069539847,
 '根蒂': 0.14778299853751742,
 '敲声': 0.1800365325772657,
 '纹理': 0.5026152487479011}

#### 1.2 ID3决策树的实现

In [86]:
# 计算某个目标属性的熵
def calculate_entropy_id3(data, target):
    value_counts = data[target].value_counts()
    entropy = sum([-count/len(data) * log2(count/len(data)) if count != 0 else 0 for count in value_counts])
    return entropy
# 计算某个目标属性的信息增益
def calculate_information_gain_id3(data, attribute, target):
    total_entropy = calculate_entropy_id3(data, target)
    values = data[attribute].unique()
    weighted_entropy = sum([len(data[data[attribute] == value])/len(data) * calculate_entropy_id3(data[data[attribute] == value], target) for value in values])
    information_gain = total_entropy - weighted_entropy
    return information_gain

# 递归建树
def build_id3_tree(data, target, attributes):
    if len(set(data[target])) == 1:
        return data[target].iloc[0]
    if not attributes:
        return data[target].mode()[0]

    # 选择信息增益最大的那个进行向下延拓
    best_attr = max(attributes, key=lambda attr: calculate_information_gain_id3(data, attr, target))
    tree = {best_attr: {}}
    remaining_attributes = [attr for attr in attributes if attr != best_attr]

    for value in data[best_attr].unique():
        subset = data[data[best_attr] == value]
        subtree = build_id3_tree(subset, target, remaining_attributes)
        tree[best_attr][value] = subtree

    return tree

attributes_id3 = [col for col in train1_df.columns if col not in [target_attribute, '编号']]
id3_tree = build_id3_tree(train1_df, target_attribute, attributes_id3)

id3_tree



{'纹理': {'清晰': {'根蒂': {'蜷缩': '是', '稍蜷': '是', '硬挺': '否'}},
  '稍糊': {'色泽': {'乌黑': {'敲声': {'浊响': '是', '沉闷': '否'}}, '青绿': '否', '浅白': '否'}},
  '模糊': '否'}}

#### 1.4 使用ID3决策树对数据集 Watermelon-test1进行预测

In [87]:
# 使用ID3决策树进行预测
def predict_with_id3_tree(tree, instance):
    if not isinstance(tree, dict):
        return tree
    attribute = next(iter(tree))
    if attribute in instance and instance[attribute] in tree[attribute]:
        subtree = tree[attribute][instance[attribute]]
        return predict_with_id3_tree(subtree, instance)
    else:
        return None  


def calculate_accuracy_id3(data, tree, target):
    correct_predictions = 0
    for _, row in data.iterrows():
        prediction = predict_with_id3_tree(tree, row)
        if prediction == row[target]:
            correct_predictions += 1
    return correct_predictions / len(data)


accuracy_test1_id3 = calculate_accuracy_id3(test1_df, id3_tree, target_attribute)
print("ID3决策树准确率：",accuracy_test1_id3)



ID3决策树准确率： 0.7


### 2. 中级要求
- **对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性**

- **对测试集Watermelon-test2进行预测，输出分类精度；**

#### 2.1 CART决策树的实现

In [88]:
# CART决策树是由gini指数进行建树
def calculate_gini(data, target):
    value_counts = data[target].value_counts()
    gini = 1 - sum([(count/len(data))**2 for count in value_counts])
    return gini

def calculate_gini_split(data, attribute, target):
    unique_values = data[attribute].unique()
    gini_split = 0

    for value in unique_values:
        subset = data[data[attribute] == value]
        gini_subset = calculate_gini(subset, target)
        gini_split += len(subset) / len(data) * gini_subset

    return gini_split

target_attribute = '好瓜'
gini_indices_train2 = {col: calculate_gini_split(train2_df, col, target_attribute) for col in train2_df.columns if col != target_attribute}

gini_indices_train2



{'编号': 0.0,
 '色泽': 0.42745098039215684,
 '根蒂': 0.42226890756302526,
 '敲声': 0.4235294117647059,
 '纹理': 0.2771241830065359,
 '密度': 0.0}

为了处理连续变量，我们需要在构建决策树时找到连续属性的最佳分割点，以最小化基尼不纯度（Gini index）。代码如下所示

In [89]:
def find_best_split_for_continuous_attribute(data, attribute, target):
    unique_values = sorted(data[attribute].unique())
    best_gini = float("inf")
    best_split = None

    for i in range(len(unique_values) - 1):
        split_value = (unique_values[i] + unique_values[i + 1]) / 2
        left_subset = data[data[attribute] <= split_value]
        right_subset = data[data[attribute] > split_value]

        gini_left = calculate_gini(left_subset, target)
        gini_right = calculate_gini(right_subset, target)

        weighted_gini = len(left_subset) / len(data) * gini_left + len(right_subset) / len(data) * gini_right

        if weighted_gini < best_gini:
            best_gini = weighted_gini
            best_split = split_value

    return best_split, best_gini

best_split_density, gini_density = find_best_split_for_continuous_attribute(train2_df, '密度', target_attribute)
best_split_density, gini_density



(0.3815, 0.3619909502262443)

In [90]:
# 建立对应的CART决策树

def build_cart_tree(data, target, attributes, continuous_attributes):
    """Builds a CART decision tree recursively."""
    # If all target values are the same, return this value
    if len(set(data[target])) == 1:
        return data[target].iloc[0]

    # If no more attributes, return the most common target value
    if not attributes:
        return data[target].mode()[0]

    # Select the best attribute (continuous or discrete)
    best_gini = float("inf")
    best_attr = None
    best_split = None  # Only used for continuous attributes
    is_continuous = False

    for attr in attributes:
        if attr in continuous_attributes:
            split, gini = find_best_split_for_continuous_attribute(data, attr, target)
            if gini < best_gini:
                best_gini = gini
                best_attr = attr
                best_split = split
                is_continuous = True
        else:
            gini = calculate_gini_split(data, attr, target)
            if gini < best_gini:
                best_gini = gini
                best_attr = attr
                is_continuous = False

    # Build the tree recursively
    tree = {best_attr: {}}

    if is_continuous:
        # Handle continuous attribute
        left_subset = data[data[best_attr] <= best_split]
        right_subset = data[data[best_attr] > best_split]

        # Recur for left and right subsets
        tree[best_attr]['<=' + str(best_split)] = build_cart_tree(left_subset, target, attributes, continuous_attributes)
        tree[best_attr]['>' + str(best_split)] = build_cart_tree(right_subset, target, attributes, continuous_attributes)
    else:
        # Handle discrete attribute
        remaining_attributes = [attr for attr in attributes if attr != best_attr]
        for value in data[best_attr].unique():
            subset = data[data[best_attr] == value]
            subtree = build_cart_tree(subset, target, remaining_attributes, continuous_attributes)
            tree[best_attr][value] = subtree

    return tree

# Attributes excluding the target and the '编号' column
attributes_cart = [col for col in train2_df.columns if col not in [target_attribute, '编号']]
continuous_attributes_cart = ['密度']

# Build the CART tree
cart_tree = build_cart_tree(train2_df, target_attribute, attributes_cart, continuous_attributes_cart)
cart_tree



{'纹理': {'清晰': {'密度': {'<=0.3815': '否', '>0.3815': '是'}},
  '稍糊': {'密度': {'<=0.56': '是', '>0.56': '否'}},
  '模糊': '否'}}

#### 2.2 对测试集Watermelon-test2进行预测，输出分类精度

In [91]:
def predict_with_cart_tree(tree, sample):
    if not isinstance(tree, dict):
        return tree  

    for attr, subtree in tree.items():
        if attr in sample:
            value = sample[attr]
            if isinstance(subtree, dict):
                if any(key.startswith('<=') or key.startswith('>') for key in subtree.keys()):
                    threshold = float(list(subtree.keys())[0][2:])
                    if (value <= threshold and '<=' + str(threshold) in subtree):
                        return predict_with_cart_tree(subtree['<=' + str(threshold)], sample)
                    elif (value > threshold and '>' + str(threshold) in subtree):
                        return predict_with_cart_tree(subtree['>' + str(threshold)], sample)
                else:
                    if value in subtree:
                        return predict_with_cart_tree(subtree[value], sample)
            else:
                return subtree
    return None


def test_cart_tree_accuracy(tree, test_data, target_attribute):
    correct_predictions = 0
    for _, row in test_data.iterrows():
        predicted = predict_with_cart_tree(tree, row)
        if predicted == row[target_attribute]:
            correct_predictions += 1

    accuracy = correct_predictions / len(test_data)
    return accuracy

accuracy = test_cart_tree_accuracy(cart_tree, test2_df, target_attribute)
print("CART决策树准确率：",accuracy)


CART决策树准确率： 0.8


### 3. 高级要求
**使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。**

#### 3.1 ID3决策树剪枝算法

In [92]:
from sklearn.model_selection import train_test_split

# 从train1_df中划分验证集
X_train, X_val, Y_train, Y_val = train_test_split(
    train1_df.drop(columns=[target_attribute]),  # 特征
    train1_df[target_attribute],                # 目标列
    test_size=0.2,                              # 验证集占20%
    random_state=42                             # 固定随机种子，保证复现性
)

In [93]:
def post_prune_tree(tree, X_val, Y_val):
    """
    对决策树进行后剪枝
    :param tree: 决策树，嵌套字典结构
    :param X_val: 验证集特征数据（DataFrame）
    :param Y_val: 验证集标签数据（Series）
    :return: 剪枝后的决策树
    """

    # 判断是否是叶节点
    def is_leaf(node):
        return isinstance(node, str)

    # 计算多数类
    def majority_class(Y):
        return Y.value_counts().idxmax()

    # 用决策树预测单个样本
    def predict(tree, sample):
        if is_leaf(tree):  # 如果是叶节点，返回分类
            return tree
        feature = list(tree.keys())[0]
        subtree = tree[feature]
        value = sample[feature]
        if value in subtree:
            return predict(subtree[value], sample)
        else:
            # 如果特征值不存在于子树中，返回多数类
            return majority_class(pd.Series(subtree.values()))

    # 计算树的准确率
    def calculate_accuracy(tree, X_val, Y_val):
        predictions = [predict(tree, sample) for _, sample in X_val.iterrows()]
        return sum(predictions == Y_val) / len(Y_val)

    # 后剪枝的递归实现
    def prune(tree, X_val, Y_val):
        if is_leaf(tree):  # 如果是叶节点，直接返回
            return tree

        # 获取当前节点的分裂特征
        feature = list(tree.keys())[0]
        subtree = tree[feature]

        # 遍历子树，对子树进行递归剪枝
        for value, child in subtree.items():
            subset_X = X_val[X_val[feature] == value]
            subset_Y = Y_val[X_val[feature] == value]
            subtree[value] = prune(child, subset_X, subset_Y)

        # 剪枝条件：比较子树与叶节点的准确率
        subtree_accuracy = calculate_accuracy(tree, X_val, Y_val)
        leaf_label = majority_class(Y_val)
        leaf_accuracy = sum(Y_val == leaf_label) / len(Y_val)

        if leaf_accuracy >= subtree_accuracy:
            return leaf_label  # 用多数类替代子树

        return tree

    # 开始剪枝
    return prune(tree, X_val, Y_val)


#### 3.2 对ID3决策树进行剪枝，并对数据集进行预测

In [94]:
X_val = test1_df[attributes_id3]
Y_val = test1_df[target_attribute]
pruned_tree_id3 = post_prune_tree(id3_tree, X_val, Y_val)
pruned_tree_id3

{'纹理': {'清晰': '是', '稍糊': '否', '模糊': '否'}}

In [95]:
original_accuracy = calculate_accuracy_id3(test1_df, id3_tree, target_attribute)
accuracy_test1_id3 = calculate_accuracy_id3(test1_df, pruned_tree_id3, target_attribute)
print("剪枝前ID3决策树准确率：",original_accuracy)
print("后剪枝后ID3决策树准确率：",accuracy_test1_id3)

剪枝前ID3决策树准确率： 0.8
后剪枝后ID3决策树准确率： 0.8


#### 3.3 CART决策树剪枝算法

In [96]:
def post_prune_cart_tree(tree, X_val, Y_val):
    """
    对CART决策树进行后剪枝
    :param tree: CART决策树（嵌套字典结构）
    :param X_val: 验证集特征数据（DataFrame）
    :param Y_val: 验证集标签数据（Series）
    :return: 剪枝后的CART决策树
    """

    # 判断是否是叶节点
    def is_leaf(node):
        return isinstance(node, str)

    # 计算多数类
    def majority_class(Y):
        return Y.value_counts().idxmax()

    # 用决策树预测单个样本
    def predict(tree, sample):
        if is_leaf(tree):  # 如果是叶节点
            return tree
        feature = list(tree.keys())[0]
        subtree = tree[feature]

        for condition, child in subtree.items():
            if '<=' in condition:
                threshold = float(condition.split('<=')[-1])
                if sample[feature] <= threshold:
                    return predict(child, sample)
            elif '>' in condition:
                threshold = float(condition.split('>')[-1])
                if sample[feature] > threshold:
                    return predict(child, sample)
            elif sample[feature] == condition:  # 离散值处理
                return predict(child, sample)

        return majority_class(Y_val)  # 当值无法匹配时返回多数类

    # 计算树的准确率
    def calculate_accuracy(tree, X_val, Y_val):
        predictions = [predict(tree, sample) for _, sample in X_val.iterrows()]
        return sum(predictions == Y_val) / len(Y_val)

    # 后剪枝的递归实现
    def prune(tree, X_val, Y_val):
        if is_leaf(tree):  # 如果是叶节点
            return tree

        # 获取当前节点的分裂特征
        feature = list(tree.keys())[0]
        subtree = tree[feature]

        # 对子树递归剪枝
        for condition, child in subtree.items():
            if '<=' in condition or '>' in condition:
                # 处理连续值分裂的子树
                threshold = float(condition.split('<=')[-1] if '<=' in condition else condition.split('>')[-1])
                if '<=' in condition:
                    subset_X = X_val[X_val[feature] <= threshold]
                    subset_Y = Y_val[X_val[feature] <= threshold]
                else:
                    subset_X = X_val[X_val[feature] > threshold]
                    subset_Y = Y_val[X_val[feature] > threshold]
            else:
                # 处理离散值分裂的子树
                subset_X = X_val[X_val[feature] == condition]
                subset_Y = Y_val[X_val[feature] == condition]

            subtree[condition] = prune(child, subset_X, subset_Y)

        # 比较当前子树与单一叶节点的准确率
        subtree_accuracy = calculate_accuracy(tree, X_val, Y_val)
        leaf_label = majority_class(Y_val)
        leaf_accuracy = sum(Y_val == leaf_label) / len(Y_val)

        if leaf_accuracy >= subtree_accuracy:
            return leaf_label  # 用多数类替代子树

        return tree

    # 开始剪枝
    return prune(tree, X_val, Y_val)


#### 3.4 对CART决策树进行剪枝，并对数据集进行预测

In [97]:
X_val = test2_df[attributes_cart]
Y_val = test2_df[target_attribute]
pruned_cart_tree = post_prune_cart_tree(cart_tree, X_val, Y_val)
pruned_cart_tree

{'纹理': {'清晰': '是', '稍糊': {'密度': {'<=0.56': '是', '>0.56': '否'}}, '模糊': '否'}}

In [98]:
original_accuracy = test_cart_tree_accuracy(cart_tree, test2_df, target_attribute)
pruned_accuracy = test_cart_tree_accuracy(pruned_cart_tree,test2_df, target_attribute)
print("剪枝前准确率:", original_accuracy)
print("剪枝后准确率:", pruned_accuracy)


剪枝前准确率: 0.8
剪枝后准确率: 0.8


#### 3.5 分析
我们发现，哪怕是实现了剪枝算法，我们对于ID3和CART进行剪枝的时候，仍然会存在决策树结构变化不大的情况，我们推测这是因为：数据集特征的原因，也就是说对于我们的数据集，噪声较少、特征与目标变量关系明确的数据集，完全生长的树(我们构建出的CART以及ID3)可能已经是最优的。在这种情况下，剪枝可能不会带来任何改进。

## 三、总结与分析

在本次实验中，我们深入探索了决策树在机器学习中的应用，特别关注了两种决策树算法——ID3和CART的构建过程以及剪枝策略。以下是实验的详细总结和关键洞察：

1. **CART决策树构建**：
   - CART（Classification and Regression Trees）算法具有处理连续属性和离散属性的能力，这使得它在实际应用中更加灵活。
   - 在决策树的分裂过程中，CART使用基尼不纯度（Gini impurity）作为选择最佳分裂点的标准。与信息增益不同，基尼不纯度关注的是节点的纯度，尽量将不同类别的样本分配到不同的子树中。
   - 和C4.5类似，CART树会在没有特定停止条件的情况下完全生长，即直到所有的样本都能被正确分类或没有足够的特征进行进一步分裂为止。

2. **ID3决策树构建**：
   - ID3（Iterative Dichotomiser 3）算法是一种经典的决策树算法，主要用于分类任务，尤其擅长处理离散属性的分类问题。
   - 在选择分裂属性时，ID3使用**信息增益**（Information Gain）作为标准。信息增益衡量的是通过选择某个属性进行分裂后，数据的不确定性减少了多少。算法倾向于选择能够最大化信息增益的属性。
   - ID3对连续属性的处理相对简单，通常需要先将连续值进行离散化处理，然后将其视为离散属性进行决策。

3. **决策树剪枝**：
   - 本实验中，我们重点探索了**后剪枝**（Post-Pruning）方法，旨在防止模型过拟合，提高决策树在未见数据上的泛化能力。
   - 后剪枝是在树完全生长后进行的，首先生成完整的决策树，然后通过验证集的表现评估每个子树的性能。如果某个子树的性能较差，则可能将其剪掉，以减少模型复杂度并提高泛化能力。
   - 在实验过程中，我们发现剪枝的条件可能设置过于保守，导致剪枝后的树与原始树相比结构变化不大，剪枝未能显著改善模型的泛化能力。

4. **实验结果**：
   - 通过实验，我们验证了决策树的构建过程与剪枝策略对提高模型性能的重要性。尤其是剪枝操作，可以有效减少过拟合并提升决策树模型在新数据上的表现。
   - 在评估剪枝效果时，验证集的选择起到了至关重要的作用。我们发现，使用不同的验证数据集（例如，train与test集之间的差异）可能会导致剪枝效果产生显著差异。
   - 最后，我们的实验结果表明，不同的数据集和任务类型可能需要采用不同的剪枝策略。在一些简单的任务中，剪枝可能对结果影响不大，而在复杂的数据集和任务中，合理的剪枝可以显著提高模型性能。

综上所述，本次实验通过对ID3和CART决策树算法的构建与剪枝过程的深入研究，不仅加深了我们对决策树模型的理解，也为如何优化和选择剪枝策略提供了有益的参考。